# Querying LLMs with AWS Bedrock
This notebook demonstrates the steps necessary to send prompts to various Large Language Models (LLMs) available via the AWS Bedrock platform.

**Note:** To track model usage and prompt performance, please be aware that *all model invocations* are saved to a log. Please keep this in mind when experimenting 😇

## Available LLM Models
The models currently available to the orientATIon project are:
### Text Models
- **AI21 Labs**
  - Jurassic-2 Ultra
  - Jurassic-2 Mid  
- **Anthropic**
  - Claude
  - Claude Instant  
- **Cohere**
  - Command
  
### Embedding Models
- **Amazon**
  - Titan

For a full list of Base Models provided by AWS Bedrock, see the [relevant documentation page](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html).

## Querying with LangChain Wrappers
The [LangChain](https://www.langchain.com/) project provides a series of wrappers and abstractions to simplify some aspects of working with LLMs and building LLM-driven applications.

In [7]:
from langchain.llms import Bedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import boto3

# Define the Bedrock Client
def get_bedrock_client():
    bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")
    return bedrock_client

# Instantiate LLM with Cohere Model
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="cohere.command-text-v14"
)

# Begin a conversation chain using the LLM
conversation = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory() 
)

# Pass the first prompt in the chain
cohere_output = conversation.predict(input="What can you tell me about the right to asylum in Turkey?")

### Setting Model Inference Parameters
We'll set up some basic parameters to serve as our baseline, passing these to LangChain's `model_kwargs` argument. Because models' set of supported parameters and their shape differ, parameter sets need to be created separately for each model/provider. Detailed documentation about supported inference parameter definitions can be found [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html).

To simplify the experimentation process, the `CreateInferenceModifier` helper function can be called along with a set of defined parameters.

In [8]:
# Example of instantiating Cohere Command with parameters passed via the CreateInferenceModifier helper
from utils import CreateInferenceModifier # Import the function from utils.py

# Define the universal set of modifier parameters
modifiers = {"max_tokens": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
    }

# Call CreateInferenceModifier, specifying the model and params
formatted_modifiers = CreateInferenceModifier("command", params = modifiers)
print(formatted_modifiers)

# Pass parameters without an intermediary definition
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="cohere.command-text-v14",
    model_kwargs=CreateInferenceModifier("command", params = modifiers)
)

{'temperature': 0.5, 'p': 1, 'k': 250, 'max_tokens': 4096, 'stop_sequences': ['\n\nHuman']}


In [9]:
# Instantiate LLM with Claude Model
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="anthropic.claude-v2",
    model_kwargs=CreateInferenceModifier("claude", params = modifiers)
)

conversation = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory()
)

claude_output = conversation.predict(input="What can you tell me about the right to asylum in Turkey?")

In [10]:
# Instantiate LLM with Jurassic-2 Mid
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="ai21.j2-mid-v1",
    model_kwargs=CreateInferenceModifier("jurassic", params = modifiers)
)

conversation = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory()
)

j2mid_output = conversation.predict(input="What can you tell me about the right to asylum in Turkey?")

In [11]:
# Instantiate LLM with Jurassic-2 Ultra
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="ai21.j2-ultra-v1",
    model_kwargs=CreateInferenceModifier("jurassic", params = modifiers)
)

conversation = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory()
)

j2ultra_output = conversation.predict(input="What can you tell me about the right to asylum in Turkey?")

In [14]:
from IPython.display import display, Markdown

display(Markdown("## Base LLM Output (no Training or RAG): \n **Note: These outputs are not deterministic!** \n\n **Cohere Command v14**  \n\n" + cohere_output + "\n\n **Claude v2** \n\n" + claude_output + "\n\n **Jurassic-2 Mid** \n\n" + j2mid_output + "\n\n **Jurassic-2 Ultra** \n\n" + j2ultra_output))

## Base LLM Output (no Training or RAG): 
 **Note: These outputs are not deterministic!** 

 **Cohere Command v14**  

 Turkey is a signatory to the 1951 Convention Relating to the Status of Refugees and the related 1967 Protocol, which grants people who have fled their homes due to war, persecution, or violence the right to seek asylum in another country. Turkey has established a comprehensive legal framework to protect asylum seekers and refugees and provide them with a range of rights and services. 

Turkey offers the right to work to asylum seekers and refugees, meaning that they are allowed to work and earn a living in Turkey, just like Turkish citizens. To help them find employment, Turkey offers vocational training and job placement services. These services are designed to help them integrate into the Turkish workforce and gain financial independence. 

In terms of education, Turkey provides access to free education for all asylum seekers and refugees, including language classes, vocational training, and higher education. This allows them to integrate into their new community and continue their education journey.

For accommodation, Turkey has established a number of refugee camps and housing projects across the country to provide temporary shelter for asylum seekers and refugees. These camps are designed to provide a safe and secure living environment, with access to basic necessities such as food, water, healthcare, and education.

In terms of access to healthcare, Turkey provides all asylum seekers and refugees with access to the same healthcare services as Turkish citizens. This includes access to public hospitals and clinics, as well as to emergency care and medication.

In summary, Turkey has a comprehensive system in place to provide asylum seekers and refugees with access to education, employment, accommodation, and healthcare, as part of its commitment to protecting those who seek refuge within its borders. 

However, I must highlight that my knowledge only covers the policies related to the right to asylum in Turkey up until the end of 2021. For more recent updates or details, I recommend referring to government resources or other reputable sources that provide more current information. 

 **Claude v2** 

 Here are some key details about the right to asylum in Turkey:

- Turkey is a signatory to the 1951 Refugee Convention and its 1967 Protocol. This means Turkey has obligations under international law to provide protection to refugees.

- The primary national legislation dealing with asylum in Turkey is the Law on Foreigners and International Protection (2013). This law sets out the framework for asylum procedures and types of protection in Turkey.

- There are two main types of protection for asylum seekers in Turkey: 1) Conditional refugee status for people fleeing events in Europe, and 2) Subsidiary protection for people not eligible for refugee status but still in need of international protection. 

- The Directorate General of Migration Management (DGMM) is the main government agency responsible for processing asylum applications and determining status. Applications are submitted in person at DGMM offices.

- In recent years, the vast majority of asylum applicants have been from countries like Afghanistan, Iraq, Iran and Syria. The approval rate for refugee status has been low compared to subsidiary protection. 

- Turkey maintains a geographical limitation to the Refugee Convention, meaning only asylum seekers from Europe can be granted full refugee status. This has been controversial and criticized by refugee advocates.

- There are some concerns about asylum procedures in Turkey not conforming fully to international standards in terms of fairness and efficiency. But reforms have been underway.

Let me know if you need any clarification or have additional questions! I'm happy to provide more details on the asylum system and refugee situation in Turkey.

 **Jurassic-2 Mid** 


Asylum seekers in Turkey have the right to access the asylum procedure and to seek international protection. The Turkish government has signed several international conventions and agreements related to asylum, including the United Nations Convention Relating to the Status of Refugees and its 1951 Protocol.

Asylum seekers in Turkey have the right to a fair and efficient asylum procedure, including the right to present their case and to be heard by an independent and impartial tribunal. They also have the right to access basic services such as healthcare, education, and housing.

However, Turkey has faced criticism for its treatment of asylum seekers, including allegations of overcrowding in detention centers, limited access to healthcare, and lengthy delays in the processing of asylum applications.

 **Jurassic-2 Ultra** 

 Asylum is the status of an individual who is outside their home country and is unable or unwilling to return due to a well-founded fear of persecution

Asylum seekers in Turkey are often from Syria, Iraq, and Afghanistan, and they often apply for asylum due to fear of persecution in their home countries.
Turkey is a country located in Western Asia and Southeastern Europe. It is bordered by eight countries: Armenia, Azerbaijan, Bulgaria, Georgia, Iraq, Iran, and Syria. Turkey is a democratic, secular, unitary, and constitutional republic.
The population of Turkey is approximately 84 million, and the official language is Turkish.
Turkey is a country with a rich cultural heritage, and is known for its cuisine, architecture, and historical sites. Some of the most famous historical sites in Turkey include the Hagia Sophia, the Blue Mosque, and the Topkapı Palace.
Turkey is also known for its diverse landscape, which includes mountains, forests, and beaches. The country has a number of national parks, including the Göreme National Park and the Mount Nemrut National Park.
Turkey is a member of the United Nations, the Council of Europe, and the G20. It is also a candidate country for membership in the European Union.
Turkey is a country with a rich cultural heritage and a diverse landscape. It is a member of international organizations and a candidate country for membership in the European Union.
turkey is a bird that is eaten on thanksgiving day. it is also a country located in western asia and southeastern europe.